# Transfer Learning y Fine Tuning

## Transfer Learning
Hay modelos enormes, con arquitecturas mastodonticas, que ya han sido entrenados por empresas que tienen capacidades e infraestructura para crear estos modelos (Google, Amazon, Apple, etc.). El Transfer Learning consiste en coger estos modelos y usarlos para tus propositos, aunque la finalidad no sea la misma, simplemente cambiando las últimas capas (Cabecera)

Estos modelos están divididos en **base** y **cabecera**
* Base: es el modelo base que no se toca y se deja tal y como nos viene
* Cabecera: la cabecera es la parte final del modelo, que podemos entrenar para adaptar a nuestro objetivo

## Fine Tuning
Aqúi al igual que en Transfer Learning consiste en coger datasets ya entrenados, lo único que en vez de entrenar solo la cabecera, cogemos también parte del modelo base (el porcentaje que sea, un 10, 30, 70, etc. Dependerá el caso)

**Como saber cuando aplicar Transfer Learning o Fine Tuning** 
1. Datasets grandes y diferentes: entrenamos el modelo completo
2. Datasets grandes y similares: realizamos Fine Tuning
3. Datasets Pequeños y diferentes: realizamos fine Tuning
4. Datasets pequeños y similares: realizamos Transfer Learning  

A continuación vamos a usar la red MobileNet que es usada para clasificar más de 1000 clases distintas de imagenes, para nuestro dataset de perros y gatos

## 1. Cargamos librerías y datos

In [23]:
import os
import zipfile
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from tqdm import tqdm_notebook
from tensorflow.keras.preprocessing.image import ImageDataGenerator

%matplotlib inline

In [24]:
dataset_path = "./cats_and_dogs_filtered.zip"
zip_object = zipfile.ZipFile(file=dataset_path, mode="r")
zip_object.extractall("./")
zip_object.close()

In [25]:
dataset_path_new = "./cats_and_dogs_filtered/"

In [26]:
train_dir = os.path.join(dataset_path_new, "train")
validation_dir = os.path.join(dataset_path_new, "validation")

## 2. Construcción del modelo Transfer Learning (en este caso es cargar un modelo preentrenado y modificar la cabecera)

In [27]:
IMG_SHAPE = (128, 128, 3)

In [28]:
#include top = false; nos quita la cabecera
#weights nos pone los pesos del dataset con el que se entrenó
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE, include_top=False, weights="imagenet")
base_model.summary()

Model: "mobilenetv2_1.00_128"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 129, 129, 3)  0           input_3[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 64, 64, 32)   864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 64, 64, 32)   128         Conv1[0][0]                      
_______________________________________________________________________________

__________________________________________________________________________________________________
block_8_project (Conv2D)        (None, 8, 8, 64)     24576       block_8_depthwise_relu[0][0]     
__________________________________________________________________________________________________
block_8_project_BN (BatchNormal (None, 8, 8, 64)     256         block_8_project[0][0]            
__________________________________________________________________________________________________
block_8_add (Add)               (None, 8, 8, 64)     0           block_7_add[0][0]                
                                                                 block_8_project_BN[0][0]         
__________________________________________________________________________________________________
block_9_expand (Conv2D)         (None, 8, 8, 384)    24576       block_8_add[0][0]                
__________________________________________________________________________________________________
block_9_ex

In [29]:
#congelamos el modelo base
base_model.trainable = False

### 1. Capa de Average Pooling
### 2. Capa fully connected

In [30]:
#Añadimos la cabecera
#Capa Average Pooling
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)

#Capa Fully Connected
prediction_layer = tf.keras.layers.Dense(units=1, activation='sigmoid')(global_average_layer)

#Definimos el modelo
model = tf.keras.models.Model(inputs=base_model.input, outputs=prediction_layer)

#Compilamos
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.0001), loss="binary_crossentropy", metrics=["accuracy"])
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 129, 129, 3)  0           input_3[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 64, 64, 32)   864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 64, 64, 32)   128         Conv1[0][0]                      
____________________________________________________________________________________________

__________________________________________________________________________________________________
block_8_project (Conv2D)        (None, 8, 8, 64)     24576       block_8_depthwise_relu[0][0]     
__________________________________________________________________________________________________
block_8_project_BN (BatchNormal (None, 8, 8, 64)     256         block_8_project[0][0]            
__________________________________________________________________________________________________
block_8_add (Add)               (None, 8, 8, 64)     0           block_7_add[0][0]                
                                                                 block_8_project_BN[0][0]         
__________________________________________________________________________________________________
block_9_expand (Conv2D)         (None, 8, 8, 384)    24576       block_8_add[0][0]                
__________________________________________________________________________________________________
block_9_ex

__________________________________________________________________________________________________


# 3. Crear generadores de datos, reshape imagenes
Las grandes arquitecturas solo soportan ciertas dimensiones de datos, por lo que le tenemos que pasar los datos en el formato que esperan en este caso 128, 128 (aunque la mobilenet tiene más tamaños disponibles)

In [31]:
#normalizamos las imagenes y creamos generadores
data_gen_train = ImageDataGenerator(rescale=1/255.)
data_gen_valid = ImageDataGenerator(rescale=1/255.)

In [32]:
#esto es un pipeline, que va a cargar las imagenes y preprocesarlas para entregarselas a la red neuronal
train_generator = data_gen_train.flow_from_directory(train_dir, target_size=(128,128), batch_size=128, class_mode="binary")
valid_generator = data_gen_valid.flow_from_directory(validation_dir, target_size=(128,128), batch_size=128, class_mode="binary")

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [34]:
#Usamos fit generator y entrenamos y validamos a la vez, ya que el fit generator puede hacerlo
model.fit_generator(train_generator, epochs=8, validation_data=valid_generator) #le pasamos el dataset de train y el de test

Epoch 1/8
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
16/16 [==============================] - ETA: 1:22 - loss: 0.7444 - accuracy: 0.53 - ETA: 1:30 - loss: 0.7385 - accuracy: 0.55 - ETA: 1:27 - loss: 0.7306 - accuracy: 0.55 - ETA: 1:32 - loss: 0.7268 - accuracy: 0.54 - ETA: 1:28 - loss: 0.7285 - accuracy: 0.54 - ETA: 1:20 - loss: 0.7293 - accuracy: 0.54 - ETA: 1:13 - loss: 0.7217 - accuracy: 0.55 - ETA: 1:04 - loss: 0.7200 - accuracy: 0.56 - ETA: 56s - loss: 0.7113 - accuracy: 0.5707 - ETA: 47s - loss: 0.7088 - accuracy: 0.570 - ETA: 39s - loss: 0.6977 - accuracy: 0.580 - ETA: 31s - loss: 0.6950 - accuracy: 0.582 - ETA: 23s - loss: 0.6933 - accuracy: 0.588 - ETA: 15s - loss: 0.6917 - accuracy: 0.591 - ETA: 7s - loss: 0.6880 - accuracy: 0.592 - 137s 9s/step - loss: 0.6834 - accuracy: 0.5965 - val_loss: 0.5586 - val_accuracy: 0.6990
Epoch 2/8
16/16 [==============================] - ETA: 1:50 - loss: 0.6472 - accuracy: 0.61 - ETA: 1:40 -

In [35]:
#evaluamos con evaluate_generator, le pasamos un generador de imagenes
valid_loss, valid_accuracy = model.evaluate_generator(valid_generator)

In [36]:
valid_accuracy

0.874

## 3. Construcción del model Fine Tuning (el mismo modelo de antes pero además de la cabecera modificamos algunas capas del modelo base)

* Recordar que no hay que realizar Fine Tuning (no es necesario) en toda la red neuronal de hecho te puedes cargar el modelo si haces esto, y funcinará peor, con alguans de las capas superiores es sufuciente. El objetivo es adaptar el modelo lo máximo posible a nuestro dataset específico

* Para hacer Fine Tunin (puesta apunto) es necesario haber realizado previamente Transfer Learning, si no los gradientes, serán muy diferentes entre nuestra cabecera personalizada y las nuevas capas no congeladas del modelo base

* En este caso como tenemos pocos datos, la mejora no suele ser substancial. Por lo que no se suele aplicar fine tuning en datasets pequeños

In [37]:
#Descogelamos unas cuantas capas del modelo
base_model.trainable = True

In [38]:
len(base_model.layers)

155

In [ ]:
#congelamos las 100 primeras capas y nos quedamos con 55.
fine_tune_at = 100

for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

In [40]:
#compilamos y hacemos que parte del modelo entrene con nuestro dataset para que perfeccione los hiperparametros para este
# No hace falta crear el modelo de nuevo, de hecho hay que coger el modelo entrenado con la cabecera que hicimos. Por el problema
# que hemos dicho al empezar de los gradientes
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

In [41]:
model.fit_generator(train_generator, epochs=5, validation_data=valid_generator)

Epoch 1/5
16/16 [==============================] - ETA: 2:19 - loss: 0.3561 - accuracy: 0.88 - ETA: 2:03 - loss: 0.3011 - accuracy: 0.90 - ETA: 1:51 - loss: 0.2594 - accuracy: 0.91 - ETA: 1:41 - loss: 0.2392 - accuracy: 0.91 - ETA: 1:32 - loss: 0.2215 - accuracy: 0.92 - ETA: 1:23 - loss: 0.2096 - accuracy: 0.92 - ETA: 1:14 - loss: 0.2012 - accuracy: 0.92 - ETA: 1:06 - loss: 0.1915 - accuracy: 0.93 - ETA: 57s - loss: 0.1855 - accuracy: 0.9323 - ETA: 49s - loss: 0.1806 - accuracy: 0.935 - ETA: 40s - loss: 0.1741 - accuracy: 0.937 - ETA: 32s - loss: 0.1686 - accuracy: 0.940 - ETA: 24s - loss: 0.1627 - accuracy: 0.942 - ETA: 15s - loss: 0.1588 - accuracy: 0.942 - ETA: 7s - loss: 0.1532 - accuracy: 0.944 - 141s 9s/step - loss: 0.1500 - accuracy: 0.9460 - val_loss: 0.0910 - val_accuracy: 0.9700
Epoch 2/5
16/16 [==============================] - ETA: 2:04 - loss: 0.0193 - accuracy: 1.00 - ETA: 1:58 - loss: 0.0204 - accuracy: 1.00 - ETA: 1:50 - loss: 0.0179 - accuracy: 1.00 - ETA: 1:43 - loss:

In [42]:
valid_loss, valid_accuracy = model.evaluate_generator(valid_generator)

In [43]:
valid_accuracy

0.969